In [1]:
import enum
import pathlib
from typing import Union, TypeAlias

import attrs
from sqlalchemy import MetaData

import vocalpy as voc

In [2]:
cd /Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy

/Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy


In [3]:
cbin_paths = voc.paths.from_dir('./tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/', 'cbin')
audio_files = [voc.dataset.AudioFile(path=cbin_path) for cbin_path in cbin_paths]

In [4]:
audio_files[:3]

[AudioFile(path=PosixPath('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin')),
 AudioFile(path=PosixPath('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0809.141.cbin')),
 AudioFile(path=PosixPath('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0810.148.cbin'))]

In [5]:
dataset_files = []
for audio_file in audio_files:
    dataset_file = voc.dataset.domain_model.DatasetFile(
        file=audio_file,
        file_type=voc.dataset.domain_model.DatasetFileTypeEnum(type(audio_file)),
        path=audio_file.path
    )
    dataset_files.append(dataset_file)

In [6]:
dataset_files[0]

DatasetFile(file=AudioFile(path=PosixPath('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin')), file_type=<DatasetFileTypeEnum.AUDIO: <class 'vocalpy.dataset.domain_model.audio_file.AudioFile'>>, path=PosixPath('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin'))

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [8]:
engine = create_engine('sqlite:///:memory:', echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
voc.dataset.orm.Base.metadata.create_all(engine)

2023-04-20 19:37:59,906 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-20 19:37:59,908 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("audio_files")
2023-04-20 19:37:59,908 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 19:37:59,909 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("audio_files")
2023-04-20 19:37:59,910 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 19:37:59,911 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("annotates")
2023-04-20 19:37:59,911 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 19:37:59,912 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("annotates")
2023-04-20 19:37:59,913 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 19:37:59,914 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("annotation_files")
2023-04-20 19:37:59,915 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-20 19:37:59,916 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("annotation_files")
2023-04-20 19:37:59,917 INFO sqlalch

In [10]:
orm_dataset_file = voc.dataset.orm.DatasetFile(
    path=str(dataset_files[0].path),
    file_type=dataset_files[0].file.__class__.__name__
)

session.add(orm_dataset_file)

In [11]:
orm_audio_file = voc.dataset.orm.AudioFile(
    dataset_file=orm_dataset_file
)

session.add(orm_audio_file)

In [12]:
session.flush()

2023-04-20 19:38:15,492 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-20 19:38:15,495 INFO sqlalchemy.engine.Engine INSERT INTO dataset_files (path, file_type) VALUES (?, ?)
2023-04-20 19:38:15,497 INFO sqlalchemy.engine.Engine [generated in 0.00147s] ('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin', 'AudioFile')
2023-04-20 19:38:15,499 INFO sqlalchemy.engine.Engine INSERT INTO audio_files (dataset_file_id) VALUES (?)
2023-04-20 19:38:15,500 INFO sqlalchemy.engine.Engine [generated in 0.00117s] (1,)


In [13]:
orm_dataset_file

DatasetFile(id=1, path='tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin', file_type='AudioFile'

In [14]:
orm_audio_file

AudioFile(id=1, dataset_file_id=1)

In [14]:
session.commit()

2023-04-20 19:16:24,184 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-20 19:16:24,192 INFO sqlalchemy.engine.Engine INSERT INTO dataset_files (path, file_type) VALUES (?, ?)
2023-04-20 19:16:24,193 INFO sqlalchemy.engine.Engine [generated in 0.00149s] ('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin', 'AudioFile')
2023-04-20 19:16:24,196 INFO sqlalchemy.engine.Engine INSERT INTO audio_files (dataset_file_id) VALUES (?)
2023-04-20 19:16:24,197 INFO sqlalchemy.engine.Engine [generated in 0.00143s] (1,)
2023-04-20 19:16:24,199 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker


def make_dataset(dataset_files, db_path='sqlite:///:memory:', echo=True):
    engine = create_engine(db_path, echo=echo)
    Session = sessionmaker(bind=engine)
    session = Session()
    
    voc.dataset.orm.Base.metadata.create_all(engine)
    
    for dataset_file in dataset_files:
        orm_dataset_file = voc.dataset.orm.DatasetFile(
            path=str(dataset_file.path),
            file_type=dataset_file.file.__name__
        )
        
        specific_file = dataset_file.file
        if isinstance(specific_file, voc.dataset.AudioFile):
            audio_file = voc.dataset.orm.AudioFile(
                

<Mapper at 0x126a29de0; DatasetFile>

In [11]:
registry.mapped

<function sqlalchemy.orm.decl_api.registry.mapped(self, cls)>

In [12]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [13]:
session.add(dataset_files[0])

UnmappedInstanceError: Class 'vocalpy.dataset.domain_model.dataset_file.DatasetFile' is mapped, but this instance lacks instrumentation.  This occurs when the instance is created before sqlalchemy.orm.mapper(vocalpy.dataset.domain_model.dataset_file.DatasetFile) was called.

In [2]:
metadata_obj = MetaData()

In [4]:
from sqlalchemy import Table, Column, Integer, String

In [8]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [ ]:
class Dataset(Base):
    __tablename__ = "dataset"
    # metadata_obj
    id = Column(Integer, primary_key=True)
    file = Column("path", String)
)